# Projeto Final - Modelos preditivos - Dataset do Censo

## Grupo:
- Lucas Natan Correia Couri
- Mariama Celi Serafim de Oliveira
- Laianna Lana Virginio da Silva
- Priscilla Amarante de Lima
- Liviany Reis Rodrigues

# Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy import stats
from sklearn import model_selection
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
filterwarnings('ignore')

# Base de Dados

In [ ]:
SEED = 6138
columns_name = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'class']
df = pd.read_csv("Dados/adult.data", names=columns_name, index_col=False)

In [ ]:
df['native-country'].value_counts()

# Análise Exploratória de Dados

explorar a base de dados para mostrar outliers, nivel de separatividade dos dados em relação as classes (grafico de dispersao), 

In [ ]:
df.head(1)

## Tipos dos dados

In [ ]:
df.dtypes

In [ ]:
df['workclass'] = df['workclass'].astype('category')
df['education'] = df['education'].astype('category')
df['marital-status'] = df['marital-status'].astype('category')
df['occupation'] = df['occupation'].astype('category')
df['relationship'] = df['relationship'].astype('category')
df['race'] = df['race'].astype('category')
df['sex'] = df['sex'].astype('category')
df['native-country'] = df['native-country'].astype('category')
df['class'] = df['class'].astype('category')
df.dtypes

## Resolvendo o problema da Holanda

No dataset de treino há apenas uma obvservação como " Holand-Netherlands", diante do tamanho do dataset (mais de 30mil linhas) optou-se por remover essa única linha com native-country=" Holand-Netherlands" de forma a evitar problemas de ausência do valor no dataset de teste.

In [ ]:
df = df[df['native-country']!=" Holand-Netherlands"]

## Descrição dos dados

In [ ]:
df.describe()

## Dados duplicados

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df[df.duplicated()]

## Preenchendo dados faltantes

In [ ]:
def tratamento_faltantes(df, columns_name):
    ## Printa os atributos com dados faltantes (" ?")
    for coluna in columns_name:
        if len(df[df[coluna] == " ?"]) > 0:
            print(coluna)
            print(len(df[df[coluna] == " ?"]))
    
    ## Tratamento dos dados faltantes, transforma para numerico, substitui " ?" por NaN e interpola os NaN
    atr_faltantes = ["workclass", "occupation", "native-country"]
    for atr in atr_faltantes:
        categorias_atr = df.groupby(atr).sum().index.tolist()
        label_encoder = preprocessing.LabelEncoder()
        label_encoder.fit(categorias_atr)
        df[f"{atr}-num"] = label_encoder.transform(df[atr])
        df[f"{atr}-num"] = df[f"{atr}-num"].replace(0, np.nan)
        df[f"{atr}-num"] = df[f"{atr}-num"].interpolate(method='nearest')

In [ ]:
tratamento_faltantes(df, columns_name)

In [ ]:
#for coluna in columns_name:
#    if len(df[df[coluna] == " ?"]) > 0:
#        print(coluna)
#        print(len(df[df[coluna] == " ?"]))

Para cada atributo que tem dados faltantes vamos preencher utilizando a interpolação, para isso passamos para numerico antes.

In [ ]:
#atr_faltantes = ["workclass", "occupation", "native-country"]
#for atr in atr_faltantes:
#    categorias_atr = df.groupby(atr).sum().index.tolist()
#    label_encoder = preprocessing.LabelEncoder()
#    label_encoder.fit(categorias_atr)
#    df[f"{atr}-num"] = label_encoder.transform(df[atr])
#    df[f"{atr}-num"] = df[f"{atr}-num"].replace(0, np.nan)
#    df[f"{atr}-num"] = df[f"{atr}-num"].interpolate(method='nearest')

In [ ]:
df.head()

## Checando outliers

In [ ]:
df['hours-per-week'].plot.box()

In [ ]:
df['hours-per-week'].hist()

In [ ]:
df['capital-gain'].plot.box()

In [ ]:
df['capital-gain'].hist()

In [ ]:
df['capital-loss'].plot.box()

In [ ]:
df['capital-loss'].hist()

In [ ]:
#q1 = dados['idade_log'].quantile(q=0.25)
#q3 = dados['idade_log'].quantile(q=0.75)
#iqr = q3 - q1
#print(iqr)

## Colunas redundantes

In [ ]:
df.head()

education e education-num significam a mesma coisa, vamos utilizar education-num e dropar education (education-num já é a codificação ordinal de education)

In [ ]:
df['education'].value_counts()

In [ ]:
df['education-num'].value_counts()

## TO DO: Plotar região

## Frequência das variáveis categóricas (Value counts)

In [ ]:
df['workclass'].value_counts()

In [ ]:
df['education'].value_counts()

In [ ]:
df['marital-status'].value_counts()

In [ ]:
df['occupation'].value_counts()

In [ ]:
df['relationship'].value_counts()

In [ ]:
df['race'].value_counts()

In [ ]:
df['sex'].value_counts()

In [ ]:
df['native-country'].value_counts()

In [ ]:
df['class'].value_counts()

In [ ]:
df_test['class'].value_counts()

# Carregando e processando conjunto de teste

In [ ]:
df_test = pd.read_csv("Dados/adult.test", names=columns_name, index_col=False, skiprows=1)
df_test.head()

In [ ]:
tratamento_faltantes(df_test, columns_name)

## Codificação das variáveis categóricas (variáveis nominais, faremos One Hot Encoder)

In [ ]:
def onehot_encoder(df):
    colunas_cat = ["workclass-num","marital-status", "occupation-num", "relationship", "race", "sex", "native-country-num"]
    for coluna in colunas_cat:
        print(coluna)
        df_coluna = pd.get_dummies(df[coluna], prefix=coluna)
        df = df.join(df_coluna)
    return df

In [ ]:
df = onehot_encoder(df)
df.head()

In [ ]:
df_test = onehot_encoder(df_test)
df_test.head()

## Normalizando variáveis contínuas

In [ ]:
from sklearn.preprocessing import MinMaxScaler

normalize = MinMaxScaler()
df[["age", "fnlwgt", "capital-gain", "capital-loss", "hours-per-week", "education-num"]] = normalize.fit_transform(df[["age", "fnlwgt", "capital-gain", "capital-loss", "hours-per-week", "education-num"]])
df_test[["age", "fnlwgt", "capital-gain", "capital-loss", "hours-per-week", "education-num"]] = normalize.fit_transform(df_test[["age", "fnlwgt", "capital-gain", "capital-loss", "hours-per-week", "education-num"]])

In [ ]:
df[["age", "fnlwgt", "capital-gain", "capital-loss", "hours-per-week", "education-num"]]

## Dividindo conjuntos de dados

In [ ]:
X_train = df.drop(["class", "education", "workclass", "workclass-num","marital-status", "occupation", "occupation-num", "relationship", "race", "sex", "native-country", "native-country-num"], axis = 1).to_numpy()
y_train = df["class"].values
X_test = df_test.drop(["class", "education", "workclass", "workclass-num","marital-status", "occupation", "occupation-num", "relationship", "race", "sex", "native-country", "native-country-num"], axis = 1).to_numpy()
y_test = df_test["class"].values

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

In [ ]:
df.drop(["class", "education", "workclass", "workclass-num","marital-status", "occupation", "occupation-num", "relationship", "race", "sex", "native-country", "native-country-num"], axis = 1)

# Testando validação

In [ ]:
#7. Realizar busca com o gridsearch ou randonsearhc para encontrar os melhores parametros de cada modelo
# define models
decisionTree = DecisionTreeClassifier()
#svc = SVC()

# define evaluation
cv = model_selection.StratifiedKFold(n_splits=10)

# define search space for decision tree
space = dict()
space['criterion'] = ['gini', 'entropy']
space['min_samples_split'] = [2,3,5,7]
space['max_depth'] = [3,5,6,7,9,11,13,15,17,19]
space['min_samples_leaf'] = [2, 3]


# define random search for decision tree
search = RandomizedSearchCV(decisionTree, space, n_iter=50, scoring='accuracy', n_jobs=-1, cv=cv, random_state=SEED)

# execute search
result_tree = search.fit(X_train, y_train)

# summarize result for decision tree
print('=========Random Search Results for TREE==========')
print('Best Score: %s' % result_tree.best_score_)
print('Best Hyperparameters: %s' % result_tree.best_params_)

In [ ]:
decisionTree = DecisionTreeClassifier(**result_tree.best_params_, random_state=SEED)

result_tree = decisionTree.fit(X_train, y_train)

print(classification_report(y_test, decisionTree.predict(X_test)))

# KNN (Livy)

# Árvore de decisão simples (Priscilla)

# Random Forest (Lucas)

## Funções

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.model_selection import cross_val_score

def f1_weighted(y_true, y_pred):
  """ Função auxiliar para o cáulo da medida-f ponderada """
  return f1_score(y_true, y_pred, average="weighted").round(3)

def score_model(model, X, y, n_splits=10, n_repeats=3, scoring='f1_weighted', random_state=SEED):

  cv = RepeatedStratifiedKFold(n_splits=n_splits, 
                               n_repeats=n_repeats, 
                               random_state=random_state)
  
  n_scores = cross_val_score(model, X, y, 
                             scoring=scoring, 
                             cv=cv, 
                             n_jobs=-1, 
                             error_score='raise')
  
  return f'{scoring.title()}: %.3f (%.3f)' % (n_scores.mean(), n_scores.std())

## Tentativa 1 (17min)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definindo a lista de parâmetros e seus possíveis valores.

parameters = {
    "n_estimators": range(10, 101, 10),
    "criterion": ["gini", "entropy"],
    "max_features": ["auto", "sqrt", "log2"],
}


# Notem que a validação agora demora mais que com os modelos que usamos até então.
grid_search = GridSearchCV(RandomForestClassifier(random_state=SEED), 
                    parameters,
                    scoring  = "f1_weighted",
                    n_jobs= -1, 
                    verbose=4,
                    cv = cv)# Quando atribuímos um número inteiro (quantidade de folds) ao parâmetro cv, a validação cruzada é estratificada

grid_search.fit(X_train, y_train)


In [ ]:
best_params = grid_search.best_params_
print(best_params)

In [ ]:
grid_search.best_score_

In [ ]:
model = RandomForestClassifier(random_state=SEED)
print("Train >>", score_model(model, X_train, y_train))
model.fit(X_train, y_train)
print("Test score: ", f1_weighted(y_test, model.predict(X_test)))

## Tentativa 2 (53min)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definindo a lista de parâmetros e seus possíveis valores.

parameters = {
    "n_estimators": range(300, 601, 10),
    "criterion": ["gini", "entropy"],
    "max_features": ["auto", "sqrt", "log2"],
}


# Notem que a validação agora demora mais que com os modelos que usamos até então.
grid_search = GridSearchCV(RandomForestClassifier(random_state=SEED), 
                    parameters,
                    scoring  = "f1_weighted",
                    n_jobs= -1, 
                    verbose=4,
                    cv = 5)# Quando atribuímos um número inteiro (quantidade de folds) ao parâmetro cv, a validação cruzada é estratificada

grid_search.fit(X_train, y_train)


In [ ]:
best_params = grid_search.best_params_
print(best_params)

In [ ]:
grid_search.best_score_

## Tentativa 3 (27min)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definindo a lista de parâmetros e seus possíveis valores.

parameters = {
    "n_estimators": range(400, 451, 3),
    "criterion": ["gini", "entropy"],
    "max_features": ["auto", "sqrt", "log2"],
}


# Notem que a validação agora demora mais que com os modelos que usamos até então.
grid_search = GridSearchCV(RandomForestClassifier(random_state=SEED), 
                    parameters,
                    scoring  = "f1_weighted",
                    n_jobs= -1, 
                    verbose=4,
                    cv = 5)# Quando atribuímos um número inteiro (quantidade de folds) ao parâmetro cv, a validação cruzada é estratificada

grid_search.fit(X_train, y_train)


In [ ]:
best_params = grid_search.best_params_
print(best_params)

In [ ]:
grid_search.best_score_

## Tentativa 4

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definindo a lista de parâmetros e seus possíveis valores.

parameters = {
    "n_estimators": range(405, 415, 1),
    "criterion": ["gini", "entropy"],
    "max_features": ["auto", "sqrt", "log2"],
}


# Notem que a validação agora demora mais que com os modelos que usamos até então.
grid_search = GridSearchCV(RandomForestClassifier(random_state=SEED), 
                    parameters,
                    scoring  = "f1_weighted",
                    n_jobs= -1, 
                    verbose=4,
                    cv = 5)# Quando atribuímos um número inteiro (quantidade de folds) ao parâmetro cv, a validação cruzada é estratificada

grid_search.fit(X_train, y_train)


In [ ]:
best_params = grid_search.best_params_
print(best_params)

In [ ]:
grid_search.best_score_

## Tentativa 5

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definindo a lista de parâmetros e seus possíveis valores.

parameters = {
    "n_estimators": range(1, 101, 1),
    "criterion": ["gini", "entropy"],
    "max_features": ["auto", "sqrt", "log2"],
}


# Notem que a validação agora demora mais que com os modelos que usamos até então.
grid_search = GridSearchCV(RandomForestClassifier(random_state=SEED), 
                    parameters,
                    scoring  = "f1_weighted",
                    n_jobs= -1, 
                    verbose=4,
                    cv = 5)# Quando atribuímos um número inteiro (quantidade de folds) ao parâmetro cv, a validação cruzada é estratificada

grid_search.fit(X_train, y_train)


In [ ]:
best_params = grid_search.best_params_
print(best_params)

In [ ]:
grid_search.best_score_

## Incluindo mais parâmetros (57min)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definindo a lista de parâmetros e seus possíveis valores.

parameters = {
    "n_estimators": [409,410,411],#range(407, 412, 1),
    "criterion": ["gini", "entropy"],
    "max_features": ["auto", "sqrt", "log2"],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'max_depth': [10, 50, 100, None]#[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]
}
  

# Notem que a validação agora demora mais que com os modelos que usamos até então.
grid_search = GridSearchCV(RandomForestClassifier(random_state=SEED), 
                    parameters,
                    scoring  = "f1_weighted",
                    n_jobs= -1, 
                    verbose=4,
                    cv = 3)# Quando atribuímos um número inteiro (quantidade de folds) ao parâmetro cv, a validação cruzada é estratificada

grid_search.fit(X_train, y_train)


In [ ]:
best_params = grid_search.best_params_
print(best_params)

In [ ]:
grid_search.best_score_

In [ ]:
def grid_searchrf(X_train, y_train, parameters, SEED, metrica, k):
    # Notem que a validação agora demora mais que com os modelos que usamos até então.
    grid_search = GridSearchCV(RandomForestClassifier(random_state=SEED), 
                    parameters,
                    scoring  = "f1_weighted",
                    n_jobs= -1, 
                    verbose=4,
                    cv = 3)# Quando atribuímos um número inteiro (quantidade de folds) ao parâmetro cv, a validação cruzada é estratificada

    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    print(best_params)
    print(grid_search.best_score_)

## Desempenho no teste

In [ ]:
model = RandomForestClassifier(**best_params, random_state = SEED)

print("Train >> ", score_model(model, X_train, y_train))
model.fit(X_train, y_train)
print("Test score pós-validação: ", f1_weighted(y_test, model.predict(X_test)))

model = RandomForestClassifier(random_state = SEED)
model.fit(X_train, y_train)
print("Test score PRÉ-validação: ", f1_weighted(y_test, model.predict(X_test)))

# Rede neural MLP (Mari)

# Comitê de Redes Neurais (Laianna)